In [2]:
# Initialize proving search agent
from src.proof_search_agent import ProvingSearchAgent
agent = ProvingSearchAgent()

## Raw Search

- Initialize the proof search agent with custom Lean code as context and return the verification result of this context.

- `Raw Search` is suitable for temporarily proving specific theorems, especially when the theorem is not included in `mathlib`.

- Similar to `Plain Search` introduced below, each proof step is a new context (generally assumed to be dealing with the same or similar problems, such as modifying a previously errored context and attempting to verify it again), returning the verification result of the newly given context. Both `Raw Search` and `Plain Search` use context as proof step and are suitable for proof search in LLM with code modification as the main function.

In [3]:
context = '''import data.nat.basic

open nat

example : 12 + 23 = 35 := sorry'''

agent.init_search_raw(context=context)
## Equivalently, you can use the following code to initialize the search agent
# init_result = agent.init_search_file(file_path='example.lean')

{'error': '',
 'warning': "line 5, column 0: declaration '[anonymous]' uses sorry",
 'info': '',
 'open_states': ['line: 5, column: 26, proof state: ⊢ 12 + 23 = 35'],
 'context': 'import data.nat.basic\n\nopen nat\n\nexample : 12 + 23 = 35 := sorry',
 'core_context': ''}

In [4]:
{c : agent.search_history.nodes[c] for c in agent.search_history.nodes()}

{'import data.nat.basic\n\nopen nat\n\nexample : 12 + 23 = 35 := sorry': {'error': '',
  'warning': "line 5, column 0: declaration '[anonymous]' uses sorry",
  'info': '',
  'open_states': ['line: 5, column: 26, proof state: ⊢ 12 + 23 = 35']}}

In [5]:
context = '''import data.nat.basic

open nat

example : 12 + 23 = 35 := by simp'''

agent.run_tac_raw(context=context)

{'error': '',
 'warning': '',
 'info': '',
 'open_states': [],
 'context': 'import data.nat.basic\n\nopen nat\n\nexample : 12 + 23 = 35 := by simp',
 'core_context': ''}

In [6]:
{c : agent.search_history.nodes[c] for c in agent.search_history.nodes()}

{'import data.nat.basic\n\nopen nat\n\nexample : 12 + 23 = 35 := sorry': {'error': '',
  'warning': "line 5, column 0: declaration '[anonymous]' uses sorry",
  'info': '',
  'open_states': ['line: 5, column: 26, proof state: ⊢ 12 + 23 = 35']},
 'import data.nat.basic\n\nopen nat\n\nexample : 12 + 23 = 35 := by simp': {'error': '',
  'warning': '',
  'info': '',
  'open_states': []}}

In [7]:
agent.close()

## Plain Search

- Initialize the proof search agent with a specific declaration in `mathlib` as context and return the verification result of this context.

In [8]:
agent = ProvingSearchAgent()
# Load proving environmnets of theorems in mathlib
agent.load_proving_envs()

init_result = agent.init_search_plain('nat.find_greatest_succ')
init_result

{'error': '',
 'warning': "line 704, column 0: declaration 'nat.find_greatest_succ' uses sorry",
 'info': '',
 'open_states': ['line: 705, column: 87, proof state: P : ℕ → Prop,\n_inst_1 : decidable_pred P,\nn : ℕ\n⊢ nat.find_greatest P (n + 1) = ite (P (n + 1)) (n + 1) (nat.find_greatest P n)'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles startin

- In the shelve file `proving_env_lib`, all public theorems (including lemmas) in the `mathlib` (revision `58a272265b5e05f258161260dd2c5d247213cbd3`) we use are extracted separately. 

- For each theorem, its proof steps are replaced with `sorry`, which we call `init_context`. All the content before the position of the theorem in the file where it is proved is extracted as `pre_lines` (we provide all the dependent content required for the proof of the theorem in this way), and the necessary commands such as closing namespaces after the theorem are extracted as `post_lines`. 

- The proof information of each theorem is stored in the shelve file as a triple `(pre_lines, init_context, post_lines)`, and the initialization of the proof environment for the theorem is to concatenate the three elements into a `.lean` code and synchronize it to the Lean server.

In [9]:
print(agent.pre_lines)

/-
Copyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.
Released under Apache 2.0 license as described in the file LICENSE.
Authors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro
-/
import order.basic
import algebra.group_with_zero.basic
import algebra.ring.defs

/-!
# Basic operations on the natural numbers

> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.
> Any changes to this file require a corresponding PR to mathlib4.

This file contains:
- instances on the natural numbers
- some basic lemmas about natural numbers
- extra recursors:
  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers
  * `decreasing_induction`: recursion growing downwards
  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions
  * `strong_rec'`: recursion based on strong inequalities
- decidability instances on predicates about the natural numbers

Many theorems that used to live in this fil

In [10]:
print(agent.init_context)

lemma find_greatest_succ (n : ℕ) :
  nat.find_greatest P (n + 1) = if P (n + 1) then n + 1 else nat.find_greatest P n :=  sorry 


In [11]:
print(agent.post_lines)


end find_greatest
end nat


In [12]:
print(init_result['context'])

/-
Copyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.
Released under Apache 2.0 license as described in the file LICENSE.
Authors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro
-/
import order.basic
import algebra.group_with_zero.basic
import algebra.ring.defs

/-!
# Basic operations on the natural numbers

> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.
> Any changes to this file require a corresponding PR to mathlib4.

This file contains:
- instances on the natural numbers
- some basic lemmas about natural numbers
- extra recursors:
  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers
  * `decreasing_induction`: recursion growing downwards
  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions
  * `strong_rec'`: recursion based on strong inequalities
- decidability instances on predicates about the natural numbers

Many theorems that used to live in this fil

In [13]:
print('\n\n'.join(init_result['open_states']))

line: 705, column: 87, proof state: P : ℕ → Prop,
_inst_1 : decidable_pred P,
n : ℕ
⊢ nat.find_greatest P (n + 1) = ite (P (n + 1)) (n + 1) (nat.find_greatest P n)


- Note that for `Plain Search`, each `proof_step` refers to a further proof of the theorem. The input of `run_tac_plain` should correspond to `init_result['context']` in the previous text, without including `pre_lines` and `post_lines`.

In [14]:
new_context = '''lemma find_greatest_succ (n : ℕ) :
  nat.find_greatest P (n + 1) = if P (n + 1) then n + 1 else nat.find_greatest P n := rfl'''
checking_result = agent.run_tac_plain(context=new_context)

In [15]:
print(checking_result['context'])


/-
Copyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.
Released under Apache 2.0 license as described in the file LICENSE.
Authors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro
-/
import order.basic
import algebra.group_with_zero.basic
import algebra.ring.defs

/-!
# Basic operations on the natural numbers

> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.
> Any changes to this file require a corresponding PR to mathlib4.

This file contains:
- instances on the natural numbers
- some basic lemmas about natural numbers
- extra recursors:
  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers
  * `decreasing_induction`: recursion growing downwards
  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions
  * `strong_rec'`: recursion based on strong inequalities
- decidability instances on predicates about the natural numbers

Many theorems that used to live in this fi

In [16]:
{c : agent.search_history.nodes[c] for c in agent.search_history.nodes()}

{"/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers\n  * `decreasing_induction`: recursion growing downwards\n  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions\n  * `strong_rec'`: recursion based on strong inequalities\n- decidability instances on predicates about the natural numbers\n\nMany theorems th

In [17]:
agent.close()

## Sequential Search

- Initialize the proof search agent with a specific declaration in `mathlib` as context and return the verification result of this context.

- Prove in the form of lean-gym by opening the tactic mode environment for the proof (replace isolated `sorry` with `begin sorry end`). 

- Each `proof step` corresponds to a lean-gym `proof step`. This is achieved by replacing the `sorry` at the end of the previous context (which must be the end of the tactic sequence between `begin` and `end`) with a tactic statement followed by `,  repeat { sorry }` (which is automatically added in the function, and there is no need to add it when inputting the tactic to the `run_tac_sequential` function).

In [18]:
agent = ProvingSearchAgent()
# Load proving environmnets of theorems in mathlib
agent.load_proving_envs()
init_result = agent.init_search_sequential('nat.find_eq_iff')
init_result

{'error': '',
 'warning': "line 651, column 0: declaration 'nat.find_eq_iff' uses sorry",
 'info': '',
 'open_states': ['line: 652, column: 17, proof state: m : ℕ,\np : ℕ → Prop,\n_inst_1 : decidable_pred p,\nh : ∃ (n : ℕ), p n\n⊢ nat.find h = m ↔ p m ∧ ∀ (n : ℕ), n < m → ¬p n'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero num

In [19]:
checking_result = agent.run_tac_sequential(tactic='\nsplit')
checking_result

{'error': '',
 'warning': "line 651, column 0: declaration 'nat.find_eq_iff' uses sorry",
 'info': '',
 'open_states': ['line: 653, column: 17, proof state: m : ℕ,\np : ℕ → Prop,\n_inst_1 : decidable_pred p,\nh : ∃ (n : ℕ), p n\n⊢ nat.find h = m → (p m ∧ ∀ (n : ℕ), n < m → ¬p n)'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero n

In [20]:
print(checking_result['core_context'])


lemma find_eq_iff (h : ∃ n : ℕ, p n) : nat.find h = m ↔ p m ∧ ∀ n < m, ¬ p n :=
 begin  
split,  repeat { sorry }   end 



In [21]:
{c : agent.search_history.nodes[c] for c in agent.search_history.nodes()}

{"/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers\n  * `decreasing_induction`: recursion growing downwards\n  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions\n  * `strong_rec'`: recursion based on strong inequalities\n- decidability instances on predicates about the natural numbers\n\nMany theorems th

In [22]:
checking_result = agent.run_tac_sequential(tactic='\nsimp')
checking_result

{'error': '',
 'warning': "line 651, column 0: declaration 'nat.find_eq_iff' uses sorry",
 'info': '',
 'open_states': ['line: 654, column: 16, proof state: m : ℕ,\np : ℕ → Prop,\n_inst_1 : decidable_pred p,\nh : ∃ (n : ℕ), p n\n⊢ nat.find h = m → (p m ∧ ∀ (n : ℕ), n < m → ¬p n)'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero n

In [23]:
print(checking_result['core_context'])


lemma find_eq_iff (h : ∃ n : ℕ, p n) : nat.find h = m ↔ p m ∧ ∀ n < m, ¬ p n :=
 begin  
split,  
simp,  repeat { sorry }    end 



In [24]:
checking_result = agent.run_tac_sequential(tactic='\n{ rintro rfl, exact ⟨nat.find_spec h, λ _, nat.find_min h⟩ }', tactic_state_id=1)
checking_result

{'error': '',
 'warning': "line 651, column 0: declaration 'nat.find_eq_iff' uses sorry",
 'info': '',
 'open_states': ['line: 654, column: 72, proof state: m : ℕ,\np : ℕ → Prop,\n_inst_1 : decidable_pred p,\nh : ∃ (n : ℕ), p n\n⊢ (p m ∧ ∀ (n : ℕ), n < m → ¬p n) → nat.find h = m'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero n

In [25]:
print(checking_result['core_context'])


lemma find_eq_iff (h : ∃ n : ℕ, p n) : nat.find h = m ↔ p m ∧ ∀ n < m, ¬ p n :=
 begin  
split,  
{ rintro rfl, exact ⟨nat.find_spec h, λ _, nat.find_min h⟩ },  repeat { sorry }    end 



In [26]:
checking_result = agent.run_tac_sequential(tactic='\nrintro ⟨hm, hlt⟩')
checking_result

{'error': '',
 'warning': "line 651, column 0: declaration 'nat.find_eq_iff' uses sorry",
 'info': '',
 'open_states': ['line: 655, column: 28, proof state: m : ℕ,\np : ℕ → Prop,\n_inst_1 : decidable_pred p,\nh : ∃ (n : ℕ), p n,\nhm : p m,\nhlt : ∀ (n : ℕ), n < m → ¬p n\n⊢ nat.find h = m'],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at n

In [27]:
print(checking_result['core_context'])


lemma find_eq_iff (h : ∃ n : ℕ, p n) : nat.find h = m ↔ p m ∧ ∀ n < m, ¬ p n :=
 begin  
split,  
{ rintro rfl, exact ⟨nat.find_spec h, λ _, nat.find_min h⟩ },  
rintro ⟨hm, hlt⟩,  repeat { sorry }     end 



In [28]:
checking_result = agent.run_tac_sequential(tactic="\nexact le_antisymm (nat.find_min' h hm) (not_lt.1 $ imp_not_comm.1 (hlt _) $ nat.find_spec h)")
checking_result

{'error': '',
 'warning': '',
 'info': '',
 'open_states': [],
 'context': "/-\nCopyright (c) 2014 Floris van Doorn (c) 2016 Microsoft Corporation. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Floris van Doorn, Leonardo de Moura, Jeremy Avigad, Mario Carneiro\n-/\nimport order.basic\nimport algebra.group_with_zero.basic\nimport algebra.ring.defs\n\n/-!\n# Basic operations on the natural numbers\n\n> THIS FILE IS SYNCHRONIZED WITH MATHLIB4.\n> Any changes to this file require a corresponding PR to mathlib4.\n\nThis file contains:\n- instances on the natural numbers\n- some basic lemmas about natural numbers\n- extra recursors:\n  * `le_rec_on`, `le_induction`: recursion and induction principles starting at non-zero numbers\n  * `decreasing_induction`: recursion growing downwards\n  * `le_rec_on'`, `decreasing_induction'`: versions with slightly weaker assumptions\n  * `strong_rec'`: recursion based on strong inequalities\n- decidabi

In [29]:
print(checking_result['core_context'])


lemma find_eq_iff (h : ∃ n : ℕ, p n) : nat.find h = m ↔ p m ∧ ∀ n < m, ¬ p n :=
 begin  
split,  
{ rintro rfl, exact ⟨nat.find_spec h, λ _, nat.find_min h⟩ },  
rintro ⟨hm, hlt⟩,  
exact le_antisymm (nat.find_min' h hm) (not_lt.1 $ imp_not_comm.1 (hlt _) $ nat.find_spec h),  repeat { sorry }      end 



In [30]:
agent.close()